In [ ]:
import os
import requests
import json
import copy
from bs4 import BeautifulSoup

In [ ]:
indicator_code = 'ER_MRN_MARIN'
start_period = '2015-01-01'

api_url = 'http://unepliveservices.unep.org/nsiws/rest/data/DF_SDG/..{}........../?startPeriod={}'.format(indicator_code, start_period)
api_url

In [ ]:
resp = requests.get(api_url)
soup = BeautifulSoup(resp.text, "xml")

In [ ]:
ref_area_lookup = None
pth = os.path.abspath('REF_AREA_LOOKUP.geojson')
with open(pth, 'rb') as file:
    ref_area_lookup = json.load(file)

In [ ]:
sdmx_fields = ['FREQ', 'REPORTING_TYPE', 'SERIES', 'REF_AREA', 'SEX', 'AGE', 'URBANISATION', 'INCOME_WEALTH_QUANTILE', 'EDUCATION_LEV', 'OCCUPATION', 'CUST_BREAKDOWN', 'DISABILITY_STATUS', 'COMPOSITE_BREAKDOWN', 'TIME_PERIOD', 'OBS_STATUS', 'UNIT_MULT', 'UNIT_MEASURE', 'NATURE', 'UPPER_BOUND', 'LOWER_BOUND', 'TIME_COVERAGE', 'SOURCE_DETAIL']

In [ ]:
geojson = {
    'type': 'FeatureCollection',
    'features': []
}

In [ ]:
def get_geometry(ref_area):   
    geom = None
    
    lookup_feature = [f for f in ref_area_lookup['features'] if f['properties']['REF_AREA'] == ref_area]
    if len(lookup_feature) > 0:
        geom = lookup_feature[0]['geometry']
        
    return geom

In [ ]:
geojson['features'] = []

ds = soup.find('DataSet')
series = ds.find_all('Series')

for s in series:
    series_key = s.find('SeriesKey')
    series_obs = s.find_all('Obs')
    sk_values = series_key.find_all('Value')
    
    properties = {}
    geom = None
    for value in sk_values:
        attributes = value.attrs
        
        if attributes['id'] == 'REF_AREA':
            geometry = get_geometry(int(attributes['value']))
            
            if geometry is not None:
                geom = geometry
            
        if attributes['id'] in sdmx_fields:
            properties[attributes['id']] = attributes['value']
    
    for obs in series_obs:
        obs_dim = obs.find('ObsDimension')
        obs_dim_field = obs_dim.attrs['id']
        obs_dim_field_value = obs_dim.attrs['value']
        
        obs_value = obs.find('ObsValue').attrs['value']
        
        obs_attributes_values = obs.find('Attributes').find_all('Value')
        
        props = copy.deepcopy(properties)
        for obs_att_value in obs_attributes_values:
            if obs_att_value['id'] in sdmx_fields:
                props[obs_att_value['id']] = obs_att_value['value']
        
        props[obs_dim_field] = obs_dim_field_value
        props['OBS_VALUE'] = obs_value
        
        feature = {
            "type": "Feature",
            "properties": props,
            "geometry": geom
        }
    
        geojson['features'].append(feature)

print (geojson)